In [1]:
import pandas as pd

# reading csv file 
df = pd.read_csv("Sample_Req.csv")

In [1]:
import os

os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
)

In [4]:
from langchain.prompts import PromptTemplate

template = '''
You are given a pandas data frame. You need to convert the data frame into JSON object.
1. Return a valid JSON. No preamble. 
2. The JSON object's keys should correspond to the column headers of the data frame, and each row should become a JSON object with its respective column values

Here is the data frame on which you need to perform this task:  
{data_frame}
'''

pt = PromptTemplate.from_template(template)

In [5]:
chain = pt | llm
response = chain.invoke(input={"data_frame": df})

In [6]:
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import JsonOutputParser

try:
    json_parser = JsonOutputParser()
    res = json_parser.parse(response.content)
except OutputParserException:
    raise OutputParserException("Context too big. Unable to parse jobs.")

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

from langchain_core.documents import Document

documents = [(Document(page_content=str(obj))) for obj in res]

In [8]:
import uuid
from langchain_chroma import Chroma

# ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

db = Chroma.from_documents(documents, embeddings_model, collection_name=unique_collection_name)

In [9]:
all_documents = db.get()["documents"]

In [10]:
from typing import Dict
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# Define the input schema
class RequirementInput(BaseModel):
    ID: str = Field(..., description="The id of the requirement, e.g., 421578")
    Primary_Text: str = Field(..., description="The primary text describing the requirement.")
    Verification_Criteria: str = Field(..., description="The verification criteria for the requirement.")
    Artifact_Type: str = Field(..., description="The type of requirement, e.g., 'SYS Requirement'.")

@tool
def write_test_case(requirement: RequirementInput) -> Dict[str, str]:
    """
       Generate Test Cases by dynamically creating 'Steps to Execute' using an LLM.
    """
    # Ensure the artifact type is correct for test case generation
    if artifact_type != "SYS Requirement":
        pass
        
    # # Extract relevant details from the requirement
    # req_ID = requirement.get("id", "No id mentioned")
    # primary_text = requirement.get("Primary Text", "No primary text provided.")
    # verification_criteria = requirement.get("Verification Criteria", "No verification criteria provided.")
    # artifact_type = requirement.get("Artifact Type", "")

    steps_prompt = (
        f"Based on the following requirement, write detailed steps to execute for testing:\n\n"
        f"Primary Text: {requirement.Primary_Text}\n"
        f"Verification Criteria: {requirement.Verification_Criteria}\n\n"
        f"Steps to Execute should be clear, actionable, and cover all necessary validation scenarios."
    )

    steps_to_execute = llm.invoke(steps_prompt).strip()

    expected_op_prompt = (
        f"For the each value of {steps_to_execute}, write its corresponding expected behaviour."
        f"Expected behaviour is to verify the input steps are correcting performed or not."
        '''Example format
        Input: 
        1. Set up the test environment to simulate Under Voltage.
        2. Verify the iPDM provides full functionality during Under Voltage.
        Expected Output:
        1. Under Voltage should be simulated
        2. iPDM should provide full functionality during Under Voltage'''
    )

    expected_op = llm.invoke(expected_op_prompt).strip()

    # Generate a test case based on the primary text
    test_case = {
        "Test Case ID": f"TC_{hash(requirement.Primary_Text) % 10000}",
        "Requirement Covered": requirement.id,
        "Objective": f"Validate: {requirement.Primary_Text}",
        "Pre-conditions": "1. The system is powered on and operational.\n"
            "2. The system is in a known state before test execution.",
        "Steps to Execute": steps_to_execute,
        "Expected Results": expected_op
    }
    return test_case

In [ ]:
# from langchain.prompts import PromptTemplate

# template = """ Role and Objective: You are an AI Test Engineer specializing in the automotive industry. Your primary responsibility is to analyze system requirements stored in a vector store, understand their context, and write detailed test cases to validate these requirements.

# How You Work:

#     Requirement Analysis:
#         Access and retrieve requirements stored in the vector store. Each requirement is in JSON format, containing fields such as:
#             Primary Text: Describes the main functionality or behavior.
#             Verification_Criteria: Specifies the conditions to validate the requirement.
#             Artifact Type: Indicates the type of requirement (e.g., "SYS Requirement"). Test cases should only be written for system requirements (Artifact Type = "SYS Requirement").

#     Context Understanding:
#         Analyze the Primary Text and the Verification_Criteria to understand the requirement's intent and scope fully.
#         Consider automotive industry standards, safety-critical elements, and typical use cases when interpreting requirements.

#     Test Case Creation:
#         Use the write_test_case tool to generate structured, clear, and concise test cases.
#         Ensure test cases:
#             Align with the Verification_Criteria.
#             Are robust and cover edge cases.
#             Follow industry-standard formats and nomenclature.

# Output Format:
# Provide test cases in a structured format. Each test case should include:

#     Test Case ID
#     Test Case Title
#     Objective
#     Pre-conditions
#     Steps
#     Expected Results

# Constraints:

#     Generate test cases only for requirements where Artifact Type is "SYS Requirement".
#     Ensure the test cases are domain-specific, practical, and implementable. 

# Example Workflow:

#     Retrieve and analyze the following requirement:

#     {
#       "Primary Text": "The system shall automatically lock all doors when the vehicle speed exceeds 10 km/h.",
#       "Verification_Criteria": "Verify that the doors lock when the vehicle speed crosses 10 km/h, and unlock only when the vehicle is stationary.",
#       "Artifact Type": "SYS Requirement"
#     }

#     Understand the context and functionality.
    
#     Write a corresponding test case:
#         Test Case ID: TC_001
#         Test Case Title: Door Lock Activation at Speed
#         Objective: Validate that the doors automatically lock when vehicle speed exceeds 10 km/h.
#         Pre-conditions: Vehicle is stationary; doors are unlocked.
#         Steps:
#         1. Start the vehicle and gradually increase the speed to 11 km/h.
#         2. Observe the door lock status.
#         3. Bring the vehicle to a complete stop.
#         4. Check the door lock status.
#         Expected Results:
#         1. At 11 km/h, all doors should lock automatically.
#         2. When the vehicle is stationary, doors should remain locked until manually unlocked.

# Key Qualities:

#     Act like a highly experienced test engineer with a deep understanding of automotive systems.
#     Be precise, systematic, and focused on producing high-quality test cases. """



In [42]:
def writetestcasetool(primary_text, verification_criteria,id):
    test_case_id = "TC_" + str(hash(primary_text))[:6]  # Generate a unique test case ID
    test_case = {
        "Test Case ID": test_case_id,
        "Requirement Coverd": id,
        "Test Case Title": f"Test for {primary_text[:30]}...",  # Shortened primary text as title
        "Objective": f"Validate that the system meets the requirement: {primary_text}",
        "Pre-conditions": "Define the system's initial state required for the test.",
        "Steps": [
            f"Step 1: {verification_criteria.split(',')[0].strip()}",
            "Step 2: Additional steps based on requirement."
        ],
        "Expected Results":  (
            f"Each test step is performed correctly, and the outcomes match the requirement's intent: {primary_text}. "
            f"Confirm proper system behavior as per verification criteria: {verification_criteria}."
        )
    }
    return json.dumps(test_case, indent=2)

In [82]:
agent_prompt_template = PromptTemplate(
    #input_variables=["requirement_json", "agent_scratchpad"],
    template=(
       "You are an AI Test Engineer for the automotive industry.\n\n"
        "Here is the system requirement to analyze:\n{input}\n\n"
        "Follow this format:\n"
        "Thought: Describe your next step in analyzing the requirement.\n"
        "Action: Indicate the tool you will use (e.g., write_test_case).\n"
        "Action_Input: Provide the input to the tool.\n"
        "Observation: Record the result of the action.\n\n"
        "Continue reasoning and repeat until the task is complete. Then provide the final test case.\n\n"
        "Current inputs:\n{input}\n\n"
        "{agent_scratchpad}"
        
    )
)

In [76]:
# llm_chain = agent_prompt_template | llm

In [77]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

In [108]:
from langchain.agents import initialize_agent, Tool

# Define the tool for writing test cases
tools = [
    Tool(
        name="write_test_case",
        description="Generate or modify structured test cases for system requirements.",
        func=lambda input: write_test_case(
        input["primary_text"], input["verification_criteria"], input["id"]
        )
    )
]
    
# Initialize the agent
test_engineer_agent = initialize_agent(
        tools,
        llm,
        prompt = agent_prompt_template,
        memory=memory, # Store interaction contex
    )

In [109]:
# Agent task execution
import json

def execute_test_case_generation(requirement):
    # requirement_json = json.dumps(requirement, indent=2)
    # input_data = {"requirement_json":  requirement, 
    #               "agent_scratchpad": "", 
    #         }
    return test_engineer_agent.invoke({"input":requirement})

In [112]:
from langchain.agents import AgentExecutor

requirement = {
    "id": "214542",
    "Primary Text": "The system shall automatically lock all doors when the vehicle speed exceeds 10 km/h",
    "Verification_Criteria": "Verify that the doors lock when the vehicle speed crosses 10 km/h, and unlock only when the vehicle is stationary.",
    "Artifact Type": "SYS Requirement"
}

# input_data = {
#     "requirement_json": json.dumps(requirement, indent=2),
#     "agent_scratchpad": ""  # Initialize scratchpad as empty
# }

agent_executor = AgentExecutor(agent=test_engineer_agent, tools=tools, verbose=True, handle_parsing_error=True)
response = agent_executor.run(input=requirement)

pprint(response)

# test_cases = execute_test_case_generation(str(requirement))
# print(test_cases)

ValueError: `run` not supported when there is not exactly one output key. Got [].

In [53]:
type(requirement)

dict